# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Project Setup

In [4]:
# all import statements needed for the project, for example:

import os

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db

In [5]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TLC_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = ""
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
WEATHER_CSV_DIR = ""

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
"""
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))
"""
DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [6]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### 1. Downloading Parquet Files

In [26]:

import requests
from bs4 import BeautifulSoup
import re
import os
from datetime import datetime

def get_parquet_links(url, regex_pattern):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = [link.get('href') for link in soup.find_all('a', href=True)]
    return [link for link in links if re.search(regex_pattern, link)]

def download_parquet_files(links, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    for link in links:
        file_name = os.path.join(save_dir, os.path.basename(link))
        response = requests.get(link, stream=True)
        with open(file_name, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"Downloaded: {file_name}")

# Filter links in date range
def filter_links_by_date(links, start_date, end_date):

    filtered_links = []
    for link in links:
        match = re.search(r'(\d{4})-(\d{2})', link)
        if match:
            year, month = int(match.group(1)), int(match.group(2))
            date = datetime(year, month, 1)
            if start_date <= date <= end_date:
                filtered_links.append(link)
    return filtered_links

# Define date range
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 8, 1)

url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
yellow_regex = r'yellow_tripdata_\d{4}-\d{2}\.parquet'
fhvhv_regex = r'fhvhv_tripdata_\d{4}-\d{2}\.parquet'

# Fetch links
yellow_links = get_parquet_links(url, yellow_regex)
fhvhv_links = get_parquet_links(url, fhvhv_regex)

# Filter links by date
yellow_links_filtered = filter_links_by_date(yellow_links, start_date, end_date)
fhvhv_links_filtered = filter_links_by_date(fhvhv_links, start_date, end_date)

# Download filtered files
download_parquet_files(yellow_links_filtered, "yellow_taxi_data")
download_parquet_files(fhvhv_links_filtered, "fhvhv_data")

Failed to download https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet : HTTP 403
Failed to download https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet : HTTP 403
Failed to download https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet : HTTP 403
Failed to download https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet  after 3 attempts.
Failed to download https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet : HTTP 403
Failed to download https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet : HTTP 403
Failed to download https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet : HTTP 403
Failed to download https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet  after 3 attempts.
Failed to download https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet : HTT

### 2. Sampling with Cochran's Formula



In [8]:
import pandas as pd
import math
import os
import glob

#### 2.1 Define the Sampling Function


In [31]:
def cochran_sample_size(population_size, confidence_level=0.95, p=0.5, margin_of_error=0.05):
    # Z-scores for common confidence levels
    z_scores = {0.9: 1.645, 0.95: 1.96, 0.99: 2.576}
    z = z_scores[confidence_level]
    
    # Cochran's initial sample size
    n_0 = (z**2 * p * (1 - p)) / (margin_of_error**2)
    
    # Adjust sample size for finite population
    if population_size > 0:
        n = n_0 / (1 + (n_0 - 1) / population_size)
    else:
        n = n_0  # Default to initial sample size if population size is unknown
    
    return math.ceil(n)

#### 2.2 Sampling for Both Datasets

In [36]:
# Function to sample a monthly dataset
def sample_monthly_data(file_path, sample_size):
    df = pd.read_parquet(file_path)
    sampled_df = df.sample(n=sample_size, random_state=42) 
    return sampled_df

# Sampling logic for Yellow Taxi and FHVHV datasets
def process_data(data_path_pattern, output_dir, dataset_type):

    os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists
    file_paths = sorted(glob.glob(data_path_pattern))  # Get all matching files
    
    for file_path in file_paths:
        # Load dataset to calculate population size
        population_size = len(pd.read_parquet(file_path))
        
        # Calculate sample size
        sample_size = cochran_sample_size(population_size, confidence_level=0.95, margin_of_error=0.05)
        print(f"{dataset_type} - {os.path.basename(file_path)}: Population size = {population_size}, Sample size = {sample_size}")
        
        # Sample data
        sampled_data = sample_monthly_data(file_path, sample_size)
        
        # Save sampled data
        output_file = os.path.join(output_dir, os.path.basename(file_path))
        sampled_data.to_parquet(output_file)
        print(f"Sampled data saved to: {output_file}")


# Process both datasets
process_data("yellow_taxi_data/yellow_tripdata_202*.parquet", "yellow_taxi_sampled_data", "Yellow Taxi")
process_data("fhvhv_data/fhvhv_tripdata_202*.parquet", "fhvhv_sampled_data", "FHVHV")

Yellow Taxi - yellow_tripdata_2020-01.parquet: Population size = 6405008, Sample size = 385
Sampled data saved to: yellow_taxi_sampled_data/yellow_tripdata_2020-01.parquet
Yellow Taxi - yellow_tripdata_2020-02.parquet: Population size = 6299367, Sample size = 385
Sampled data saved to: yellow_taxi_sampled_data/yellow_tripdata_2020-02.parquet
Yellow Taxi - yellow_tripdata_2020-03.parquet: Population size = 3007687, Sample size = 385
Sampled data saved to: yellow_taxi_sampled_data/yellow_tripdata_2020-03.parquet
Yellow Taxi - yellow_tripdata_2020-04.parquet: Population size = 238073, Sample size = 384
Sampled data saved to: yellow_taxi_sampled_data/yellow_tripdata_2020-04.parquet
Yellow Taxi - yellow_tripdata_2020-05.parquet: Population size = 348415, Sample size = 384
Sampled data saved to: yellow_taxi_sampled_data/yellow_tripdata_2020-05.parquet
Yellow Taxi - yellow_tripdata_2020-06.parquet: Population size = 549797, Sample size = 384
Sampled data saved to: yellow_taxi_sampled_data/yel

#### 2.3 Cleaning and Filtering

In [1]:
import os
import pandas as pd
import geopandas as gpd
import glob
import math
import requests
import zipfile
from bs4 import BeautifulSoup
import re

# Coordinate Reference System
CRS = 4326

# (lat, lon) bounding box
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

# Define Uber-specific license number and base numbers
uber_license_num = "HV0003"
uber_base_numbers = [
    "B02877", "B02866", "B02882", "B02869", "B02617", "B02876", "B02865", "B02512", 
    "B02888", "B02864", "B02883", "B02875", "B02682", "B02880", "B02870", "B02404", 
    "B02598", "B02765", "B02879", "B02867", "B02878", "B02887", "B02872", "B02836", 
    "B02884", "B02835", "B02764", "B02889", "B02871", "B02395"
]

In [40]:
#!pip install fastparquet
#!pip install pyarrow

# Load Taxi Zone Shapefile
taxi_zones = gpd.read_file("taxi_zones.shp")

# Project to a planar CRS before calculating centroids
projected_taxi_zones = taxi_zones.to_crs(epsg=3857)  # Web Mercator projection
projected_taxi_zones["centroid"] = projected_taxi_zones.geometry.centroid

# Reproject centroids back to geographic CRS for latitude/longitude extraction
centroids_geographic = projected_taxi_zones["centroid"].to_crs(epsg=4326)
taxi_zones["latitude"] = centroids_geographic.y
taxi_zones["longitude"] = centroids_geographic.x

# Create lookup table for LocationID to Latitude/Longitude
location_lookup = taxi_zones[["LocationID", "latitude", "longitude"]]

# Load Weather Data
weather_files = [
    "/Users/zhulilan/Documents/GitHub/final_project01/weather_data/2020_weather.csv",
    "/Users/zhulilan/Documents/GitHub/final_project01/weather_data/2021_weather.csv",
    "/Users/zhulilan/Documents/GitHub/final_project01/weather_data/2022_weather.csv",
    "/Users/zhulilan/Documents/GitHub/final_project01/weather_data/2023_weather.csv",
    "/Users/zhulilan/Documents/GitHub/final_project01/weather_data/2024_weather.csv"
]

# Define data type mapping for weather columns
dtype_mapping = {
    "HourlyPrecipitation": "string",
    "DailySnowfall": "string",
    "DailyPrecipitation": "string",
    "LATITUDE": "float",
    "LONGITUDE": "float",
    # Add other columns as needed
}

# Load and concatenate weather data with specified data types
weather_data = pd.concat(
    [pd.read_csv(file, dtype=dtype_mapping, low_memory=False) for file in weather_files],
    ignore_index=True
)

# Ensure numeric columns are converted to float after handling mixed types
numeric_columns = ["HourlyPrecipitation", "DailySnowfall", "DailyPrecipitation"]
for col in numeric_columns:
    weather_data[col] = pd.to_numeric(weather_data[col], errors="coerce").fillna(0)


# Keep only the relevant columns
weather_columns_to_keep = ["DATE", "LATITUDE", "LONGITUDE", "HourlyPrecipitation", "HourlyWindSpeed", "DailySnowfall", "DailyPrecipitation", "DailyAverageWindSpeed"]
weather_data = weather_data[weather_columns_to_keep]

# Rename columns to maintain consistency
weather_data.rename(columns={
    "DATE": "date",
    "LATITUDE": "latitude",
    "LONGITUDE": "longitude",
    "HourlyPrecipitation": "precipitation_hourly",
    "DailyPrecipitation": "precipitation_daily",
    "HourlyWindSpeed": "wind_speed_hourly",
    "DailyAverageWindSpeed": "wind_speed_daily",
    "DailySnowfall": "snowfall"
}, inplace=True)

# Convert 'T' values to 0.005 in relevant columns
weather_data[['precipitation_hourly', 'precipitation_daily', 'snowfall']] = weather_data[['precipitation_hourly', 'precipitation_daily', 'snowfall']].replace('T', 0.005)

# Remove any alphabetic characters from numeric columns
weather_data[['precipitation_hourly', 'precipitation_daily', 'snowfall']] = weather_data[['precipitation_hourly', 'precipitation_daily', 'snowfall']].replace(r'[a-zA-Z]', '', regex=True)

# Convert date column to datetime
weather_data['date'] = pd.to_datetime(weather_data['date'])

# Fill any missing values with zeros (for precipitation, wind speed, snowfall)
weather_data[['precipitation_hourly', 'precipitation_daily', 'wind_speed_hourly', 'wind_speed_daily', 'snowfall']] = weather_data[['precipitation_hourly', 'precipitation_daily', 'wind_speed_hourly', 'wind_speed_daily', 'snowfall']].fillna(0)

# Ensure correct data types
weather_data['precipitation_hourly'] = weather_data['precipitation_hourly'].astype(float)
weather_data['precipitation_daily'] = weather_data['precipitation_daily'].astype(float)
weather_data['wind_speed_hourly'] = weather_data['wind_speed_hourly'].astype(float)
weather_data['wind_speed_daily'] = weather_data['wind_speed_daily'].astype(float)
weather_data['snowfall'] = weather_data['snowfall'].astype(float)
weather_data['latitude'] = weather_data['latitude'].astype(float)
weather_data['longitude'] = weather_data['longitude'].astype(float)

# Save cleaned weather data to a separate file
weather_output_file = "cleaned_weather_data.parquet"
weather_data.to_parquet(weather_output_file)
print(f"Cleaned weather data saved to: {weather_output_file}")

Cleaned weather data saved to: cleaned_weather_data.parquet


In [38]:

def clean_and_filter_data(df, location_lookup, dataset_type, weather_data):
    """
    Cleans and filters the ride data, and merges with weather data.
    """

     # Filter Uber rides for FHVHV dataset
    if dataset_type == "FHVHV":
        # Normalize column values by converting to lowercase and stripping whitespaces
        df['hvfhs_license_num'] = df['hvfhs_license_num'].str.lower().str.strip()
        df['dispatching_base_num'] = df['dispatching_base_num'].str.lower().str.strip()
        
        # Define Uber-specific license number and base numbers
        uber_license_num = "hv0003"  # Make sure it's in lowercase
        """
        uber_base_numbers = [
            "b02877", "b02866", "b02882", "b02869", "b02617", "b02876", "b02865", "b02512", 
            "b02888", "b02864", "b02883", "b02875", "b02682", "b02880", "b02870", "b02404", 
            "b02598", "b02765", "b02879", "b02867", "b02878", "b02887", "b02872", "b02836", 
            "b02884", "b02835", "b02764", "b02889", "b02871", "b02395"
        ]
        """
        # Filter for Uber rides only
        df = df[
            (df['hvfhs_license_num'] == uber_license_num)]
        #&
          #  (df['dispatching_base_num'].isin(uber_base_numbers))
        
        

    """
        print("Cleaned DataFrame (after filtering uber rides):")
        print(df.head())
    """
    
    # Merge LocationID with Latitude/Longitude
    df = df.merge(location_lookup, left_on="PULocationID", right_on="LocationID", how="left")
    df = df.merge(location_lookup, left_on="DOLocationID", right_on="LocationID", how="left", suffixes=("_pickup", "_dropoff"))

    
    # Drop rows with missing latitude/longitude
    df = df.dropna(subset=["latitude_pickup", "longitude_pickup", "latitude_dropoff", "longitude_dropoff"])
    """
    print("Cleaned DataFrame (before removing unnecessary):")
    print(df.head())
    """

    # Remove unnecessary columns based on project queries
    if dataset_type == "Yellow Taxi":
        columns_to_keep = [
            "tpep_pickup_datetime", "tpep_dropoff_datetime",
            "latitude_pickup", "longitude_pickup", "latitude_dropoff", "longitude_dropoff", 
            "fare_amount", "trip_distance", "total_amount", "tip_amount"
        ]
        df = df[columns_to_keep]

        # Rename columns for Yellow Taxi
        df.rename(columns={
            "latitude_pickup": "latitude_pickup1",
            "longitude_pickup": "longitude_pickup1",
            "latitude_dropoff": "latitude_dropoff1",
            "longitude_dropoff": "longitude_dropoff1"
        }, inplace=True)

    elif dataset_type == "FHVHV":
        columns_to_keep = [
            "pickup_datetime", "dropoff_datetime",
            "latitude_pickup", "longitude_pickup", "latitude_dropoff", "longitude_dropoff", 
            "base_passenger_fare", "trip_miles", "tolls", "tips"
        ]
        df = df[columns_to_keep]

        # Rename columns for FHVHV
        df.rename(columns={
            "latitude_pickup": "latitude_pickup2",
            "longitude_pickup": "longitude_pickup2",
            "latitude_dropoff": "latitude_dropoff2",
            "longitude_dropoff": "longitude_dropoff2"
        }, inplace=True)
    

    
    # Remove trips with zero distance
    #df = df[(df["latitude_pickup"] != df["latitude_dropoff"]) & (df["longitude_pickup"] != df["longitude_dropoff"])]

    if dataset_type == "Yellow Taxi":
        df = df[
            (df["latitude_pickup1"] != df["latitude_dropoff1"]) &
            (df["longitude_pickup1"] != df["longitude_dropoff1"])
        ]
    elif dataset_type == "FHVHV":
        df = df[
            (df["latitude_pickup2"] != df["latitude_dropoff2"]) &
            (df["longitude_pickup2"] != df["longitude_dropoff2"])
        ]
    #print(f"Columns available before filtering: {df.columns.tolist()}")
    

    # Remove trips outside of the NYC bounding box
    def is_within_bounding_box(lat, lon, bounding_box):
        return (bounding_box[0][0] <= lat <= bounding_box[1][0]) and (bounding_box[0][1] <= lon <= bounding_box[1][1])
    
    if dataset_type == "Yellow Taxi":
        df = df[df.apply(lambda row: is_within_bounding_box(row["latitude_pickup1"], row["longitude_pickup1"], NEW_YORK_BOX_COORDS) and
                                    is_within_bounding_box(row["latitude_dropoff1"], row["longitude_dropoff1"], NEW_YORK_BOX_COORDS), axis=1)]
    elif dataset_type == "FHVHV":
        df = df[df.apply(lambda row: is_within_bounding_box(row["latitude_pickup2"], row["longitude_pickup2"], NEW_YORK_BOX_COORDS) and
                                    is_within_bounding_box(row["latitude_dropoff2"], row["longitude_dropoff2"], NEW_YORK_BOX_COORDS), axis=1)]

    # Normalize column names
    df.columns = df.columns.str.lower()
    
    return df


# Paths for Yellow Taxi and FHVHV data
yellow_taxi_path_pattern = "yellow_taxi_sampled_data/yellow_tripdata_202*.parquet"
fhvhv_path_pattern = "fhvhv_sampled_data/fhvhv_tripdata_202*.parquet"

# Output directories
yellow_taxi_output_dir = "yellow_taxi_cleaned_data"
fhvhv_output_dir = "fhvhv_cleaned_data"

# Process both datasets
def process_cleaning(data_path_pattern, output_dir, dataset_type, weather_data):
    os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists
    file_paths = sorted(glob.glob(data_path_pattern))  # Get all matching files
    
    for file_path in file_paths:
        df = pd.read_parquet(file_path)
        cleaned_data = clean_and_filter_data(df, location_lookup, dataset_type, weather_data)
        output_file = os.path.join(output_dir, os.path.basename(file_path))
        cleaned_data.to_parquet(output_file)
        print(f"Cleaned data saved to: {output_file}")

# Run the cleaning process
process_cleaning(yellow_taxi_path_pattern, yellow_taxi_output_dir, "Yellow Taxi", weather_data)
process_cleaning(fhvhv_path_pattern, fhvhv_output_dir, "FHVHV", weather_data)



Cleaned data saved to: yellow_taxi_cleaned_data/yellow_tripdata_2020-01.parquet
Cleaned data saved to: yellow_taxi_cleaned_data/yellow_tripdata_2020-02.parquet
Cleaned data saved to: yellow_taxi_cleaned_data/yellow_tripdata_2020-03.parquet
Cleaned data saved to: yellow_taxi_cleaned_data/yellow_tripdata_2020-04.parquet
Cleaned data saved to: yellow_taxi_cleaned_data/yellow_tripdata_2020-05.parquet
Cleaned data saved to: yellow_taxi_cleaned_data/yellow_tripdata_2020-06.parquet
Cleaned data saved to: yellow_taxi_cleaned_data/yellow_tripdata_2020-07.parquet
Cleaned data saved to: yellow_taxi_cleaned_data/yellow_tripdata_2020-08.parquet
Cleaned data saved to: yellow_taxi_cleaned_data/yellow_tripdata_2020-09.parquet
Cleaned data saved to: yellow_taxi_cleaned_data/yellow_tripdata_2020-10.parquet
Cleaned data saved to: yellow_taxi_cleaned_data/yellow_tripdata_2020-11.parquet
Cleaned data saved to: yellow_taxi_cleaned_data/yellow_tripdata_2020-12.parquet
Cleaned data saved to: yellow_taxi_clean

In [12]:
file_path = "/Users/vivianwang/Desktop/IEOR4501/Final Project/cleaned_weather_data.parquet"
df = pd.read_parquet(file_path)
df

,date,latitude,longitude,precipitation_hourly,wind_speed_hourly,snowfall,precipitation_daily,wind_speed_daily
0,2020-01-01 00:51:00,40.77898,-73.96925,0.0,8.0,0.0,0.0,0.0
1,2020-01-01 01:51:00,40.77898,-73.96925,0.0,8.0,0.0,0.0,0.0
2,2020-01-01 02:51:00,40.77898,-73.96925,0.0,14.0,0.0,0.0,0.0
3,2020-01-01 03:51:00,40.77898,-73.96925,0.0,11.0,0.0,0.0,0.0
4,2020-01-01 04:51:00,40.77898,-73.96925,0.0,6.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
56093,2024-10-22 14:51:00,40.77898,-73.96925,0.0,3.0,0.0,0.0,0.0
56094,2024-10-22 15:51:00,40.77898,-73.96925,0.0,0.0,0.0,0.0,0.0
56095,2024-10-22 16:51:00,40.77898,-73.96925,0.0,0.0,0.0,0.0,0.0
56096,2024-10-22 17:51:00,40.77898,-73.96925,0.0,0.0,0.0,0.0,0.0


## Part 2: Storing Cleaned Data

### 2.1 Define table schemas using SQL

In [1]:

HOURLY_WEATHER_SCHEMA = """
CREATE TABLE hourly_weather (
    date TEXT,
    latitude REAL,
    longitude REAL,
    precipitation_hourly REAL,
    wind_speed_hourly REAL
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE daily_weather (
    date TEXT,
    latitude REAL,
    longitude REAL,
    precipitation_daily REAL,
    snowfall REAL,
    wind_speed_daily REAL
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE taxi_trips (
    pickup_datetime TEXT,
    dropoff_datetime TEXT,
    latitude_pickup REAL,
    longitude_pickup REAL,
    latitude_dropoff REAL,
    longitude_dropoff REAL,
    fare_amount REAL,
    trip_distance REAL,
    total_amount REAL,
    tip_amount REAL
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE uber_trips (
    pickup_datetime TEXT,
    dropoff_datetime TEXT,
    latitude_pickup REAL,
    longitude_pickup REAL,
    latitude_dropoff REAL,
    longitude_dropoff REAL,
    base_passenger_fare REAL,
    trip_miles REAL,
    tolls REAL,
    tips REAL
);
"""


In [3]:
# Write the schemas to a schema.sql file
DATABASE_SCHEMA_FILE = "schema.sql"

with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

print(f"Schema written to {DATABASE_SCHEMA_FILE}")


Schema written to schema.sql


### 2.2 Add Data to Database

In [5]:
import glob
import pandas as pd
from sqlalchemy import create_engine

# Define database URL and schema file
DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"

# Combine Yellow Taxi Parquet files
yellow_taxi_files = glob.glob("yellow_taxi_cleaned_data/*.parquet")
taxi_data = pd.concat([pd.read_parquet(file) for file in yellow_taxi_files], ignore_index=True)
print(f"Loaded {len(taxi_data)} Yellow Taxi records.")

# Combine Uber Parquet files
uber_files = glob.glob("fhvhv_cleaned_data/*.parquet")
uber_data = pd.concat([pd.read_parquet(file) for file in uber_files], ignore_index=True)
print(f"Loaded {len(uber_data)} Uber records.")

# Load Weather Data
weather_data = pd.read_parquet("cleaned_weather_data.parquet")

# Split Weather Data
hourly_columns = ["date", "latitude", "longitude", "precipitation_hourly", "wind_speed_hourly"]
hourly_data = weather_data[hourly_columns].dropna(subset=["precipitation_hourly", "wind_speed_hourly"])

daily_columns = ["date", "latitude", "longitude", "precipitation_daily", "snowfall", "wind_speed_daily"]
daily_data = weather_data[daily_columns].dropna(subset=["precipitation_daily", "snowfall", "wind_speed_daily"])

# Map table names to DataFrames
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

# Create SQLite Engine
engine = create_engine(DATABASE_URL)

# Function to write DataFrames to tables
def write_dataframes_to_table(table_to_df_dict):
    with engine.connect() as connection:
        for table_name, df in table_to_df_dict.items():
            print(f"Writing data to table: {table_name}")
            df.to_sql(table_name, connection, if_exists="replace", index=False)
            print(f"Data written to table: {table_name}")

# Write DataFrames to SQLite database
write_dataframes_to_table(map_table_name_to_dataframe)
"""
from sqlalchemy.sql import text

# Verify data in the database
with engine.connect() as connection:
    for table in map_table_name_to_dataframe.keys():
        # Wrap the query in text() to make it executable
        query = text(f"SELECT COUNT(*) FROM {table}")
        result = connection.execute(query).fetchone()
        print(f"Table {table} contains {result[0]} records.")

    # Optionally, preview a sample of each table
    for table in map_table_name_to_dataframe.keys():
        query = text(f"SELECT * FROM {table} LIMIT 5")
        sample = pd.read_sql(query, connection)
        print(f"Sample data from {table}:")
        print(sample)
"""

Loaded 20120 Yellow Taxi records.
Loaded 13756 Uber records.
Writing data to table: taxi_trips
Data written to table: taxi_trips
Writing data to table: uber_trips
Data written to table: uber_trips
Writing data to table: hourly_weather
Data written to table: hourly_weather
Writing data to table: daily_weather
Data written to table: daily_weather


'\nfrom sqlalchemy.sql import text\n\n# Verify data in the database\nwith engine.connect() as connection:\n    for table in map_table_name_to_dataframe.keys():\n        # Wrap the query in text() to make it executable\n        query = text(f"SELECT COUNT(*) FROM {table}")\n        result = connection.execute(query).fetchone()\n        print(f"Table {table} contains {result[0]} records.")\n\n    # Optionally, preview a sample of each table\n    for table in map_table_name_to_dataframe.keys():\n        query = text(f"SELECT * FROM {table} LIMIT 5")\n        sample = pd.read_sql(query, connection)\n        print(f"Sample data from {table}:")\n        print(sample)\n'

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

### Query 1

#### query 1

In [32]:
from sqlalchemy import create_engine
import pandas as pd

# Connect to the SQLite database
DATABASE_URL = "sqlite:///project.db"
engine = create_engine(DATABASE_URL)

query_1 = """
SELECT 
    strftime('%H', tpep_pickup_datetime) AS hour_of_day,
    COUNT(*) AS ride_count
FROM 
    taxi_trips
GROUP BY 
    hour_of_day
ORDER BY 
    ride_count DESC
"""

# Write the query to a .sql file
with open("popular_taxi_hours.sql", "w") as file:
    file.write(query_1)

# Execute the query and load results into a DataFrame
with engine.connect() as connection:
    results = pd.read_sql(query_1, connection)

results



,hour_of_day,ride_count
0,18,1446
1,17,1406
2,15,1360
3,16,1302
4,14,1291
5,13,1187
6,19,1186
7,12,1171
8,21,1034
9,20,1015


#### query2

In [34]:
# Connect to the SQLite database
DATABASE_URL = "sqlite:///project.db"
engine = create_engine(DATABASE_URL)

query_2 = """
SELECT 
    strftime('%w', pickup_datetime) AS day_of_week,
    CASE strftime('%w', pickup_datetime)
        WHEN '0' THEN 'Sunday'
        WHEN '1' THEN 'Monday'
        WHEN '2' THEN 'Tuesday'
        WHEN '3' THEN 'Wednesday'
        WHEN '4' THEN 'Thursday'
        WHEN '5' THEN 'Friday'
        WHEN '6' THEN 'Saturday'
    END AS day_name,
    COUNT(*) AS ride_count
FROM 
    uber_trips
GROUP BY 
    day_of_week
ORDER BY 
    ride_count DESC
"""

# Write the query to a .sql file
with open("popular_uber_days.sql", "w") as file:
    file.write(query_2)

# Execute the query
with engine.connect() as connection:
    results = pd.read_sql(query_2, connection)

results


,day_of_week,day_name,ride_count
0,6,Saturday,2342
1,5,Friday,2192
2,4,Thursday,1966
3,0,Sunday,1892
4,3,Wednesday,1876
5,2,Tuesday,1806
6,1,Monday,1682


#### query 3

In [38]:
# Connect to the SQLite database
DATABASE_URL = "sqlite:///project.db"
engine = create_engine(DATABASE_URL)

query_3 = """
WITH ordered_distances AS (
    SELECT trip_distance
    FROM taxi_trips
    WHERE strftime('%Y-%m', tpep_pickup_datetime) = '2024-01'
    UNION ALL
    SELECT trip_miles AS trip_distance
    FROM uber_trips
    WHERE strftime('%Y-%m', pickup_datetime) = '2024-01'
),
ranked_distances AS (
    SELECT 
        trip_distance,
        ROW_NUMBER() OVER (ORDER BY trip_distance) AS row_num,
        COUNT(*) OVER () AS total_rows
    FROM ordered_distances
)
SELECT 
    trip_distance AS distance_95_percentile
FROM ranked_distances
WHERE row_num = CAST(0.95 * total_rows AS INTEGER);
"""

# Write the query to a .sql file
with open("trip_distance_95_percentile.sql", "w") as file:
    file.write(query_3)



# Execute the query
with engine.connect() as connection:
    results = connection.execute(query_3).fetchone()

# Display the results
print("95% Percentile of Trip Distance in January 2024:", results[0])


95% Percentile of Trip Distance in January 2024: 16.04


### query 4

In [ ]:
query_4 = """WITH all_rides_2023 AS (
    SELECT 
        DATE(tpep_pickup_datetime) AS ride_date,
        COUNT(*) AS total_rides,
        AVG(trip_distance) AS avg_distance
    FROM taxi_trips
    WHERE DATE(tpep_pickup_datetime) BETWEEN '2023-01-01' AND '2023-12-31'
    GROUP BY ride_date

    UNION ALL

    SELECT 
        DATE(pickup_datetime) AS ride_date,
        COUNT(*) AS total_rides,
        AVG(trip_miles) AS avg_distance
    FROM uber_trips
    WHERE DATE(pickup_datetime) BETWEEN '2023-01-01' AND '2023-12-31'
    GROUP BY ride_date
),
weather_aggregates AS (
    SELECT 
        DATE(date) AS weather_date,
        AVG(precipitation_daily) AS avg_precipitation,
        AVG(wind_speed_daily) AS avg_wind_speed
    FROM daily_weather
    WHERE DATE(date) BETWEEN '2023-01-01' AND '2023-12-31'
    GROUP BY weather_date
)
SELECT 
    a.ride_date AS date,
    SUM(a.total_rides) AS total_rides,
    AVG(a.avg_distance) AS avg_distance,
    w.avg_precipitation,
    w.avg_wind_speed
FROM all_rides_2023 a
LEFT JOIN weather_aggregates w
    ON a.ride_date = w.weather_date
GROUP BY a.ride_date
ORDER BY total_rides DESC
LIMIT 10;
"""

# Save to a .sql file
with open("busiest_days_weather.sql", "w") as f:
    f.write(query_4)

In [ ]:
with open("busiest_days_weather.sql", "r") as f:
    query_4 = f.read()

with engine.connect() as connection:
    result_4 = connection.execute(text(query_4)).fetchall()
    
# Convert to DataFrame
columns_4 = ["date", "total_rides", "avg_distance", "avg_precipitation", "avg_wind_speed"]
df_query_4 = pd.DataFrame(result_4, columns=columns_4)
print("Query 4 Results:")
print(df_query_4)

### query 5

In [ ]:
query_5 = """
WITH all_hired_rides AS (
    SELECT 
        DATE(tpep_pickup_datetime) AS ride_date,
        COUNT(*) AS total_rides
    FROM taxi_trips
    WHERE DATE(tpep_pickup_datetime) BETWEEN '2020-01-01' AND '2024-08-31'
    GROUP BY ride_date

    UNION ALL

    SELECT 
        DATE(pickup_datetime) AS ride_date,
        COUNT(*) AS total_rides
    FROM uber_trips
    WHERE DATE(pickup_datetime) BETWEEN '2020-01-01' AND '2024-08-31'
    GROUP BY ride_date
),
daily_snowfall AS (
    SELECT 
        DATE(date) AS snow_date,
        SUM(snowfall) AS total_snowfall
    FROM daily_weather
    WHERE snowfall > 0 AND DATE(date) BETWEEN '2020-01-01' AND '2024-08-31'
    GROUP BY snow_date
)
SELECT 
    s.snow_date AS date,
    s.total_snowfall,
    COALESCE(SUM(r.total_rides), 0) AS total_rides
FROM daily_snowfall s
LEFT JOIN all_hired_rides r
    ON s.snow_date = r.ride_date
GROUP BY s.snow_date, s.total_snowfall
ORDER BY s.total_snowfall DESC
LIMIT 10;
"""

# Save to a .sql file
with open("snow_days.sql", "w") as f:
    f.write(query_5)

In [ ]:
with open("snow_days.sql", "r") as f:
    query_5 = f.read()

with engine.connect() as connection:
    result_5 = connection.execute(text(query_5)).fetchall()

# Convert to DataFrame
columns_5 = ["date", "total_snowfall", "total_rides"]
df_query_5 = pd.DataFrame(result_5, columns=columns_5)
print("Query 5 Results:")
print(df_query_5)

### query 6

In [ ]:
query_6 = """
WITH all_hired_rides_hourly AS (
    SELECT 
        STRFTIME('%Y-%m-%d %H:00:00', tpep_pickup_datetime) AS hour,
        COUNT(*) AS total_rides
    FROM taxi_trips
    WHERE DATE(tpep_pickup_datetime) BETWEEN '2023-09-25' AND '2023-10-03'
    GROUP BY hour

    UNION ALL

    SELECT 
        STRFTIME('%Y-%m-%d %H:00:00', pickup_datetime) AS hour,
        COUNT(*) AS total_rides
    FROM uber_trips
    WHERE DATE(pickup_datetime) BETWEEN '2023-09-25' AND '2023-10-03'
    GROUP BY hour
),
weather_hourly AS (
    SELECT 
        STRFTIME('%Y-%m-%d %H:00:00', date) AS hour,
        SUM(precipitation_hourly) AS total_precipitation,
        AVG(wind_speed_hourly) AS avg_wind_speed
    FROM hourly_weather
    WHERE DATE(date) BETWEEN '2023-09-25' AND '2023-10-03'
    GROUP BY hour
),
all_hours AS (
    SELECT DISTINCT
        STRFTIME('%Y-%m-%d %H:00:00', date) AS hour
    FROM hourly_weather
    WHERE DATE(date) BETWEEN '2023-09-25' AND '2023-10-03'
    UNION
    SELECT DISTINCT
        STRFTIME('%Y-%m-%d %H:00:00', tpep_pickup_datetime) AS hour
    FROM taxi_trips
    WHERE DATE(tpep_pickup_datetime) BETWEEN '2023-09-25' AND '2023-10-03'
    UNION
    SELECT DISTINCT
        STRFTIME('%Y-%m-%d %H:00:00', pickup_datetime) AS hour
    FROM uber_trips
    WHERE DATE(pickup_datetime) BETWEEN '2023-09-25' AND '2023-10-03'
)
SELECT 
    h.hour,
    COALESCE(r.total_rides, 0) AS total_rides,
    COALESCE(w.total_precipitation, 0.0) AS total_precipitation,
    COALESCE(w.avg_wind_speed, 0.0) AS avg_wind_speed
FROM all_hours h
LEFT JOIN all_hired_rides_hourly r ON h.hour = r.hour
LEFT JOIN weather_hourly w ON h.hour = w.hour
WHERE DATE(h.hour) BETWEEN '2023-09-25' AND '2023-10-03'  -- Restrict to exact range
ORDER BY h.hour ASC;
"""

with open("ophelia.sql", "w") as f:
    f.write(query_6)

In [ ]:
# Execute the query
with open("ophelia_fixed.sql", "w") as f:
    f.write(query_6)

with engine.connect() as connection:
    result_6 = connection.execute(text(query_6)).fetchall()

# Convert to DataFrame
columns_6 = ["hour", "total_rides", "total_precipitation", "avg_wind_speed"]
df_query_6 = pd.DataFrame(result_6, columns=columns_6)
print(df_query_6)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)